In [3]:
import math
import time

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
from torch.nn.modules.utils import _pair
from collections import namedtuple


class SelfAttentionConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size,
                 stride=1, padding=0, groups=1, bias=True):
        print('in_channels, out_channels, kernel_size, stride, padding, groups')
        print(in_channels, out_channels, kernel_size, stride, padding, groups)
        super(SelfAttentionConv2d, self).__init__()
        if in_channels % groups != 0:
            raise ValueError('in_channels must be divisible by groups')
        if out_channels % groups != 0:
            raise ValueError('out_channels must be divisible by groups')
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = _pair(kernel_size)
        self.stride = _pair(stride)
        self.padding = _pair(padding)
        self.groups = groups  # multi-head count
        self.weight = namedtuple('fakeweight', ['shape'])([self.out_channels, self.in_channels])

        if bias:
            self.bias = nn.Parameter(torch.Tensor(1, out_channels, 1, 1))
        else:
            self.register_parameter('bias', None)

        # relative position offsets are shared between multi-heads
        self.rel_size = (out_channels) // 2
        self.relative_x = nn.Parameter(torch.Tensor(
            self.rel_size, self.kernel_size[1]))
        self.relative_y = nn.Parameter(torch.Tensor(
            (out_channels) - self.rel_size, self.kernel_size[0]))

        self.weight_query = nn.Conv2d(self.in_channels, self.out_channels,
                                      1, groups=self.groups, bias=False)
        self.weight_key = nn.Conv2d(self.in_channels, self.out_channels,
                                    1, groups=self.groups, bias=False)
        self.weight_value = nn.Conv2d(self.in_channels, self.out_channels,
                                      1, groups=self.groups, bias=False)

        self.softmax = nn.Softmax(dim=3)

        self.reset_parameters()

    def reset_parameters(self):
        init.kaiming_normal_(self.weight_query.weight, mode='fan_out', nonlinearity='relu')
        init.kaiming_normal_(self.weight_key.weight, mode='fan_out', nonlinearity='relu')
        init.kaiming_normal_(self.weight_value.weight, mode='fan_out', nonlinearity='relu')

        if self.bias is not None:
            bound = 1 / math.sqrt(self.out_channels)
            init.uniform_(self.bias, -bound, bound)

        init.normal_(self.relative_x, 0, 1)
        init.normal_(self.relative_y, 0, 1)

    def forward(self, x):
        b, c, h, w = x.size()
        kh, kw = self.kernel_size
        ph, pw = h + self.padding[0] * 2, w + self.padding[1] * 2

        fh = (ph - kh) // self.stride[0] + 1
        fw = (pw - kw) // self.stride[1] + 1

        px, py = self.padding
        x = F.pad(x, (py, py, px, px))

        vq = self.weight_query(x)
        vk = self.weight_key(x)
        vv = self.weight_value(x)  # b, fc, ph, pw

        # b, fc, fh, fw
        win_q = vq[:, :, (kh - 1) // 2:ph - (kh // 2):self.stride[0],
                   (kw - 1) // 2:pw - (kw // 2):self.stride[1]]

        win_q_b = win_q.view(b, -1, fh, fw)  # b, g, fc/g, fh, fw

        # (b, g, x, fh, fw), (b, g, y, fh, fw)
        win_q_x, win_q_y = win_q_b.split(self.rel_size, dim=1)
        win_q_x = torch.einsum('bxhw,xk->bhwk', (win_q_x, self.relative_x))  # b, fh, fw, kw
        win_q_y = torch.einsum('byhw,yk->bhwk', (win_q_y, self.relative_y))  # b, fh, fw, kh

        win_k = vk.unfold(2, kh, self.stride[0]).unfold(
            3, kw, self.stride[1])  # b, fc, fh, fw, kh, kw

        vx = (win_q.unsqueeze(4).unsqueeze(4) * win_k).sum(dim=1)  # b, fh, fw, kh, kw
        vx = vx + win_q_x.unsqueeze(3) + win_q_y.unsqueeze(4)  # add rel_x, rel_y
        vx = self.softmax(vx.view(b, fh, fw, -1)).view(b, 1, fh, fw, kh, kw)

        win_v = vv.unfold(2, kh, self.stride[0]).unfold(3, kw, self.stride[1])
        # (b, fc, fh, fw, kh, kw) -> (b, fc, fh, fw)
        fin_v = torch.einsum('bchwkl->bchw', (vx * win_v, ))

        if self.bias is not None:
            fin_v += self.bias

        return fin_v

# l = SelfAttentionConv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, groups=1)
# l = SelfAttentionConv2d(16, 32, 3, stride=1, padding=1, groups=4)
# x = torch.randn(3, 16, 8, 8)
# l(x).shape

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        Conv2dUse = nn.Conv2d
        Conv2dUse = SelfAttentionConv2d
        self.conv1 = nn.Conv2d(1, 16, 5, 1)
        self.bn1 = nn.BatchNorm2d(self.conv1.weight.shape[0])
        self.conv2 = Conv2dUse(16, 32, 5, 1, padding=2)
        self.bn2 = nn.BatchNorm2d(self.conv2.weight.shape[0])
        self.conv3 = Conv2dUse(32, 64, 5, 1, padding=2)
        self.bn3 = nn.BatchNorm2d(self.conv3.weight.shape[0])
        self.drop = nn.Dropout(0.5)
        self.fc1 = nn.Linear(64*3*3, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.avg_pool2d(x, 2, 2)
        x = self.bn1(x)
        x = F.relu(self.conv2(x))
        x = F.avg_pool2d(x, 2, 2)
        x = self.bn2(x)
        x = F.relu(self.conv3(x))
        x = F.avg_pool2d(x, 2, 2)
        x = self.bn3(x)
        x = x.view(-1, 64*3*3)
        x = self.drop(x)
        x = self.fc1(x)
        return F.log_softmax(x, dim=1)
    
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

args = {
    'batch_size': 64,
    'test_batch_size': 1000,
    'epochs': 10,
    'lr': 0.01,
    'momentum': 0.5,
    'no_cuda': False,
    'seed': 1,
    'log_interval': 100,
    'save_model': False
}
args = namedtuple("args", args.keys())(*args.values())

use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size, shuffle=True, **kwargs)


model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)

for epoch in range(1, args.epochs + 1):
    train(args, model, device, train_loader, optimizer, epoch)
    test(args, model, device, test_loader)

if (args.save_model):
    torch.save(model.state_dict(),"mnist_cnn.pt")

in_channels, out_channels, kernel_size, stride, padding, groups
16 32 5 1 2 1
in_channels, out_channels, kernel_size, stride, padding, groups
32 64 5 1 2 1


RuntimeError: CUDA error: out of memory